In [32]:
import numpy as np
import json
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
from generate_dataset import word_set
from gvsm import perform_gvsm

In [33]:
file_paths = glob.glob('data/dataset_*.json')

all_data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        all_data.append(data)


In [34]:
def preprocess(data):
    i =0
    result = []
    doc_strings = [" ".join(doc) for doc in data["documents"]+ [data["query"]]]
    vectorizer = TfidfVectorizer(vocabulary=word_set)
    tfidf_matrix = vectorizer.fit_transform(doc_strings).toarray()

    actual_document = tfidf_matrix[:-1]
    actual_query = tfidf_matrix[-1]

    for x in actual_document:
        mx = list(x)
        for q in actual_query:
            mx.append(q)
        y = data["gvsm_ranked_docs"][i]
        result.append({"input":mx, "output":y})
        i+=1
    return result

In [35]:
X = []
Y = []

for data in all_data:
    result = preprocess(data)
    for r in result:
        X.append(r["input"])
        Y.append(r["output"])

X = np.array(X)
Y = np.array(Y)

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [36]:

import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
model = models.Sequential([
    ## fixed input
    layers.Input(shape=(len( X[0] ),)),           # 64 input features
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1,activation='relu') 
])

# mae cuz 0-1
model.compile(optimizer='adam', loss='mae')
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,281 (5.00 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# --- Train the Model ---
# Using validation_data so that the test set is evaluated at the end of each epoch
model.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test))

Epoch 1/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - loss: 0.2304 - val_loss: 0.0213
Epoch 2/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 278us/step - loss: 0.0205 - val_loss: 0.0207
Epoch 3/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 286us/step - loss: 0.0191 - val_loss: 0.0193
Epoch 4/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 291us/step - loss: 0.0186 - val_loss: 0.0199
Epoch 5/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 282us/step - loss: 0.0183 - val_loss: 0.0184
Epoch 6/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - loss: 0.0190 - val_loss: 0.0194
Epoch 7/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 288us/step - loss: 0.0177 - val_loss: 0.0197
Epoch 8/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 313us/step - loss: 0.0180 - val_loss: 0.0183
Epoch 9/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 315us/step - loss: 0.0175 - val_loss: 0.0190
Epoch 10/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 296us/step - loss: 0.0178 - val_loss: 0.0181
Epoch 11/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - loss: 0.0181 - val_loss: 0.0205
Epoch 12/100
271/27

In [38]:
# --- Evaluate on the Test Set ---
test_loss = model.evaluate(X_test, Y_test)
print("Test loss:", test_loss)

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 212us/step - loss: 0.0163
Test loss: 0.01802566461265087


## Test your doc

In [39]:
mystuff = {
    "documents":[
        ['bird', 'cat', 'bird', 'cat', 'dog', 'dog', 'bird'],
        ['cat', 'tiger', 'cat', 'dog'],
        ['dog', 'bird', 'bird'],
        ['cat', 'tiger'],
        ['tiger', 'tiger', 'dog', 'tiger', 'cat'],
        ['cat', 'cat', 'tiger', 'tiger'],
        ['bird', 'cat', 'dog'],
        ['dog', 'cat', 'bird'],
        ['cat', 'dog', 'tiger'],
        ['tiger', 'cat', 'tiger'],
        ['cat'],
        ['cat'],
        ['cat'],
        ['cat'],
        ['cat']
    ],
    "query":["cat","tiger","tiger","tiger"],
    "gvsm_ranked_docs":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]
}

model_input = preprocess(mystuff)
cleanX = []
for x in model_input:
    cleanX.append(x["input"])
predicted_cosim = model.predict(np.array(cleanX))

ranked_docs = pd.DataFrame({
    "Document": [i+1 for i in range(len(predicted_cosim))],
    "CosineSimilarity": [cosim[0] for cosim in predicted_cosim],
}).sort_values(by="CosineSimilarity", ascending=False).reset_index(drop=True)
print("OUR MODEL")
print(ranked_docs)

print("REAL GVSM")
perform_gvsm({str(i+1): doc for i, doc in enumerate(mystuff["documents"])}, mystuff["query"])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
OUR MODEL
    Document  CosineSimilarity
0          5          1.002657
1          9          1.002542
2          2          0.993010
3         10          0.991542
4          4          0.991033
5          6          0.991033
6         11          0.975766
7         14          0.975766
8         13          0.975766
9         15          0.975766
10        12          0.975766
11         7          0.966001
12         8          0.966001
13         1          0.957414
14         3          0.952767
REAL GVSM

Ranked Documents by Cosine Similarity:
    Document  CosineSimilarity  Minterm
0        10          0.997369        4
1         5          0.979960        2
2         4          0.975516        4
3         6          0.975516        4
4         9          0.900074        2
5         2          0.888183        2
6        11          0.798895        5
7        14          0.798895        5
8        13          0.798895        5
9        15    

/home/ss/repos/PYTHON/KMITL/websearch/gvsm.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return freq_table.applymap(lambda x: int(x > 0))


array([0.48652646, 0.88818328, 0.25778659, 0.97551597, 0.97995983,
       0.97551597, 0.54665779, 0.54665779, 0.90007367, 0.99736876,
       0.79889477, 0.79889477, 0.79889477, 0.79889477, 0.79889477])